### **title 기반 Consistency (일관성- 분산 측정) 설계**

같은 입력값에 노래는 달라도 분위기는 일관성 있어야함

- **Goal:** 같은 상황(`Home` + `Focus`)을 10번 물어봤을 때, 추천된 노래들의 '결(Vibe)'이 비슷한가?

- **Method:** 

1. Main KPI: Vibe Coherence (LLM Judge)

실제 노래와 primary_tag가 일관적으로 나오는지 LLM이 판단

목표: 평균 4.0점 이상 (5점 만점)

의미: 10곡의 플레이리스트가 하나의 테마로 묶일 수 있어야 함.

Fail 조건 (1~2점): 중간에 튀는 곡(Outlier)이 2곡 이상 섞여 있을 때.

예: 자장가 9곡 + 헤비메탈 1곡 (이건 0점 처리해야 함).


2. Sub KPI: Tag Consistency (Rule-based)

primary_tag의 빈도

목표: Dominant Tag 비율 70% 이상

의미: 프롬프트가 안정적으로 작동한다는 증거.

해석: 점수가 낮더라도 Vibe Score가 높다면, GPT가 **'유의어(Synonym)'**를 쓰고 있는 것이므로 큰 문제는 아님.



- **Target:** 표준편차가 낮을수록 좋음 (들쑥날쑥하지 않음)= Tag Consistency가 높음 = Vibe Coherence가 높음 .

In [ ]:
import pandas as pd
from openai import OpenAI
import json
import time
from collections import Counter

# ===========================================================
# 1. 설정
# ===========================================================
API_KEY = ""
client = OpenAI(api_key=API_KEY)

OUTPUT_FILE = "consistency_result.xlsx"

# 테스트할 시나리오 (대표 3종)
TEST_CASES = [
    {"Location": "library", "Goal": "focus", "Decibel": "silent", "Pref": "Piano"},
    {"Location": "gym", "Goal": "active", "Decibel": "loud", "Pref": "Pop"},
    {"Location": "home", "Goal": "sleep", "Decibel": "quiet", "Pref": "Ballad"}
]

ITERATIONS = 10  # 케이스당 반복 횟수

# ===========================================================
# 2. 프롬프트 (최종 스키마 사용)
# ===========================================================
SYSTEM_PROMPT_DJ = """
# Role
당신은 사용자의 상황(Context)과 취향(Preference)에 맞춰, Spotify에 실존하는 음악을 추천하는 AI DJ입니다.

# Critical Constraints
1. **Real Song Only**: 반드시 실제로 존재하는 곡을 추천해야 합니다.
2. **Context Alignment**: 목표(Goal)와 소음(Decibel)을 최우선으로 고려하세요.
3. **Primary Tag**: 추천한 곡의 분위기를 가장 잘 설명하는 태그(mood_genre)를 하나만 적으세요. (예: focus_piano, workout_pop)

# Output JSON Structure
{
  "recommendation_meta": {
    "reasoning": "...",
    "primary_tag": "mood_tag_here"
  },
  "track_info": {
    "artist_name": "Artist",
    "track_title": "Title"
  },
  "target_audio_features": { ... }
}
"""

SYSTEM_PROMPT_JUDGE = """
# Role
당신은 음악 플레이리스트의 통일성을 평가하는 전문가입니다.

# Task
주어진 10곡의 리스트(Artist - Title)와 태그들을 보고, '분위기의 일관성(Consistency)'을 평가하세요.

# Input
- Context: {context}
- Song List: 10 songs generated by AI

# Scoring (1-5)
- 5점: 10곡 모두 분위기가 완벽하게 통일됨.
- 3점: 1~2곡 정도 분위기가 약간 다름.
- 1점: 리스트가 중구난방임 (예: 자장가와 메탈이 섞여 있음).

# Output JSON
{"score": int, "reason": "string"}
"""

# ===========================================================
# 3. 함수 정의
# ===========================================================
def get_recommendation(case):
    """ GPT에게 추천 요청 """
    user_msg = f"Location: {case['Location']}, Goal: {case['Goal']}, Pref: {case['Pref']}"
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_DJ},
                {"role": "user", "content": user_msg}
            ],
            response_format={"type": "json_object"},
            temperature=0.7 # 창의성을 위해 0.7 유지 (매번 다른 곡이 나와야 하므로)
        )
        return json.loads(response.choices[0].message.content)
    except:
        return {}

def evaluate_coherence(case, songs_history):
    """ 10곡을 모아서 한번에 일관성 평가 """
    song_list_str = "\n".join([f"- {s['artist']} - {s['title']} (Tag: {s['tag']})" for s in songs_history])
    
    judge_msg = f"""
    [Context] {case['Location']} / {case['Goal']}
    
    [Generated Playlist (10 runs)]
    {song_list_str}
    
    이 리스트가 해당 상황에서 일관성 있게 재생될 수 있는지 평가해줘.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_JUDGE},
                {"role": "user", "content": judge_msg}
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)
    except:
        return {"score": 0, "reason": "Error"}

# ===========================================================
# 4. 메인 실행
# ===========================================================
print(f" Consistency(일관성) 평가 시작... (총 {len(TEST_CASES)}개 시나리오 x {ITERATIONS}회)")
final_results = []

for case in TEST_CASES:
    scenario_name = f"{case['Location']} / {case['Goal']}"
    print(f"\n▶ Testing Scenario: {scenario_name}")
    
    songs_history = []
    tags_history = []
    
    # 1. 10번 반복 생성 (Simulation)
    for i in range(ITERATIONS):
        res = get_recommendation(case)
        
        track = res.get('track_info', {})
        meta = res.get('recommendation_meta', {})
        
        artist = track.get('artist_name', 'Unknown')
        title = track.get('track_title', 'Unknown')
        tag = meta.get('primary_tag', 'Unknown')
        
        songs_history.append({'artist': artist, 'title': title, 'tag': tag})
        tags_history.append(tag)
        print(f"  [{i+1}/{ITERATIONS}] {artist} - {title} ({tag})")
        time.sleep(0.3)
        
    # 2. 지표 1: Tag Consistency 계산
    # 가장 많이 나온 태그의 빈도수 / 전체 횟수
    tag_counts = Counter(tags_history)
    most_common_tag, count = tag_counts.most_common(1)[0]
    tag_consistency_score = (count / ITERATIONS) * 100
    
    # 3. 지표 2: Vibe Coherence (LLM Judge)
    judge_result = evaluate_coherence(case, songs_history)
    
    print(f"   Tag Consistency: {tag_consistency_score}% (Dominant: {most_common_tag})")
    print(f"   Vibe Score: {judge_result['score']}/5 ({judge_result['reason']})")
    
    final_results.append({
        "Scenario": scenario_name,
        "Tag Consistency (%)": tag_consistency_score,
        "Dominant Tag": most_common_tag,
        "Vibe Score (1-5)": judge_result['score'],
        "Judge Reason": judge_result['reason'],
        "Generated Songs": str([f"{s['artist']} - {s['title']}" for s in songs_history])
    })

# 결과 저장
df = pd.DataFrame(final_results)
df.to_excel(OUTPUT_FILE, index=False)
print(f"\n 평가 완료! {OUTPUT_FILE} 저장됨.")

 Consistency(일관성) 평가 시작... (총 3개 시나리오 x 10회)

▶ Testing Scenario: library / focus
  [1/10] Ludovico Einaudi - Nuvole Bianche (focus_piano)
  [2/10] Ludovico Einaudi - Nuvole Bianche (focus_piano)
  [3/10] Ludovico Einaudi - Nuvole Bianche (focus_piano)
  [4/10] Ludovico Einaudi - Nuvole Bianche (focus_piano)
  [5/10] Yiruma - River Flows in You (focus_piano)
  [6/10] Ludovico Einaudi - Nuvole Bianche (focus_piano)
  [7/10] Ludovico Einaudi - Nuvole Bianche (focus_piano)
  [8/10] Ludovico Einaudi - Nuvole Bianche (focus_piano)
  [9/10] Ludovico Einaudi - Nuvole Bianche (focus_piano)
  [10/10] Ludovico Einaudi - Nuvole Bianche (focus_piano)
   Tag Consistency: 100.0% (Dominant: focus_piano)
   Vibe Score: 5/5 (플레이리스트에 있는 모든 곡이 피아노 중심의 클래식 분위기를 가지고 있어서, 'focus'와 관련된 분위기를 잘 유지하고 있습니다. 또한, 9곡이 같은 곡으로 반복되고 있는 점에서 매우 높은 일관성을 보여줍니다. 두 곡 모두 같은 아티스트의 작품이기 때문에, 연속 재생 시 분위기가 여전히 유지됩니다.)

▶ Testing Scenario: gym / active
  [1/10] Dua Lipa - Physical (workout_pop)
  [2/10] Dua Lipa - Don't Start Now

GPT가 자유롭게 primary_tag를 출력함

---

In [ ]:
import pandas as pd
from openai import OpenAI
import json
import time
from collections import Counter

# ===========================================================
# 1. 설정
# ===========================================================
API_KEY = ""
client = OpenAI(api_key=API_KEY)

OUTPUT_FILE = "어휘제한_consistency_result.xlsx"

# 테스트할 시나리오 (대표 3종)
TEST_CASES = [
    {"Location": "library", "Goal": "focus", "Decibel": "silent", "Pref": "Piano"},
    {"Location": "gym", "Goal": "active", "Decibel": "loud", "Pref": "Pop"},
    {"Location": "home", "Goal": "sleep", "Decibel": "quiet", "Pref": "Ballad"}
]

ITERATIONS = 10  # 케이스당 반복 횟수

# ===========================================================
# 2. 프롬프트 (최종 스키마 사용)
# ===========================================================
SYSTEM_PROMPT_DJ = """
# Role
당신은 사용자의 상황(Context)과 취향(Preference)에 맞춰, Spotify에 실존하는 음악을 추천하는 AI DJ입니다.

# Critical Constraints
1. **Real Song Only**: 반드시 실제로 존재하는 곡을 추천해야 합니다.
2. **Context Alignment**: 목표(Goal)와 소음(Decibel)을 최우선으로 고려하세요.

#  Primary Tag Generation Rules (Strict)
To facilitate data analysis, you must generate the 'primary_tag' following this strict format:

1. **Format**: `"{Goal}_{Genre}"` (Snake case)
2. **Prefix (Goal)**: Use the exact 'Goal' from the user input.
   - Options: focus, relax, sleep, active, anger, consolation, neutral
3. **Suffix (Genre)**: Choose the most representative genre from this list ONLY.
   - Options: pop, k-pop, rock, hip-hop, r-nb, jazz, classical, electronic, lo-fi, ambient, soundtrack, acoustic, metal, indie, folk
4. **Example**:
   - Correct: `focus_classical`, `active_rock`, `sleep_ambient`
   - Incorrect: `study_piano` (Wrong Goal), `relax_chill` (Vague Genre), `anger_hardcore` (Genre not in list)

# Output JSON Structure (Must Follow)
응답은 반드시 아래 JSON 포맷을 정확히 지켜야 하며, 다른 말은 포함하지 마세요.

{
  "recommendation_meta": {
    "reasoning": "추천 사유 (한글 1문장)",
    "primary_tag": "Goal_Genre format (e.g., focus_lofi)"
  },
  "track_info": {
    "artist_name": "Exact Artist Name",
    "track_title": "Exact Song Title"
  },
  "target_audio_features": {
     "min_tempo": 80,
     "max_tempo": 100,
     "target_energy": 0.5,
     "target_instrumentalness": 0.9,
     "target_valence": 0.5,
     "target_acousticness": 0.4 
  }
} """

SYSTEM_PROMPT_JUDGE = """
# Role
당신은 음악 플레이리스트의 통일성을 평가하는 전문가입니다.

# Task
주어진 10곡의 리스트(Artist - Title)와 태그들을 보고, '분위기의 일관성(Consistency)'을 평가하세요.

# Input
- Context: {context}
- Song List: 10 songs generated by AI

# Scoring (1-5)
- 5점: 10곡 모두 분위기가 완벽하게 통일됨.
- 3점: 1~2곡 정도 분위기가 약간 다름.
- 1점: 리스트가 중구난방임 (예: 자장가와 메탈이 섞여 있음).

# Output JSON
{"score": int, "reason": "string"}

"""

# ===========================================================
# 3. 함수 정의
# ===========================================================
def get_recommendation(case):
    """ GPT에게 추천 요청 """
    user_msg = f"Location: {case['Location']}, Goal: {case['Goal']}, Pref: {case['Pref']}"
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_DJ},
                {"role": "user", "content": user_msg}
            ],
            response_format={"type": "json_object"},
            temperature=0.7 # 창의성을 위해 0.7 유지 (매번 다른 곡이 나와야 하므로)
        )
        return json.loads(response.choices[0].message.content)
    except:
        return {}

def evaluate_coherence(case, songs_history):
    """ 10곡을 모아서 한번에 일관성 평가 """
    song_list_str = "\n".join([f"- {s['artist']} - {s['title']} (Tag: {s['tag']})" for s in songs_history])
    
    judge_msg = f"""
    [Context] {case['Location']} / {case['Goal']}
    
    [Generated Playlist (10 runs)]
    {song_list_str}
    
    이 리스트가 해당 상황에서 일관성 있게 재생될 수 있는지 평가해줘.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_JUDGE},
                {"role": "user", "content": judge_msg}
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)
    except:
        return {"score": 0, "reason": "Error"}

# ===========================================================
# 4. 메인 실행
# ===========================================================
print(f" Consistency(일관성) 평가 시작... (총 {len(TEST_CASES)}개 시나리오 x {ITERATIONS}회)")
final_results = []

for case in TEST_CASES:
    scenario_name = f"{case['Location']} / {case['Goal']}"
    print(f"\n▶ Testing Scenario: {scenario_name}")
    
    songs_history = []
    tags_history = []
    
    # 1. 10번 반복 생성 (Simulation)
    for i in range(ITERATIONS):
        res = get_recommendation(case)
        
        track = res.get('track_info', {})
        meta = res.get('recommendation_meta', {})
        
        artist = track.get('artist_name', 'Unknown')
        title = track.get('track_title', 'Unknown')
        tag = meta.get('primary_tag', 'Unknown')
        
        songs_history.append({'artist': artist, 'title': title, 'tag': tag})
        tags_history.append(tag)
        print(f"  [{i+1}/{ITERATIONS}] {artist} - {title} ({tag})")
        time.sleep(0.3)
        
    # 2. 지표 1: Tag Consistency 계산
    # 가장 많이 나온 태그의 빈도수 / 전체 횟수
    tag_counts = Counter(tags_history)
    most_common_tag, count = tag_counts.most_common(1)[0]
    tag_consistency_score = (count / ITERATIONS) * 100
    
    # 3. 지표 2: Vibe Coherence (LLM Judge)
    judge_result = evaluate_coherence(case, songs_history)
    
    print(f"   Tag Consistency: {tag_consistency_score}% (Dominant: {most_common_tag})")
    print(f"   Vibe Score: {judge_result['score']}/5 ({judge_result['reason']})")
    
    final_results.append({
        "Scenario": scenario_name,
        "Tag Consistency (%)": tag_consistency_score,
        "Dominant Tag": most_common_tag,
        "Vibe Score (1-5)": judge_result['score'],
        "Judge Reason": judge_result['reason'],
        "Generated Songs": str([f"{s['artist']} - {s['title']}" for s in songs_history])
    })

# 결과 저장
df = pd.DataFrame(final_results)
df.to_excel(OUTPUT_FILE, index=False)
print(f"\n 평가 완료! {OUTPUT_FILE} 저장됨.")

 Consistency(일관성) 평가 시작... (총 3개 시나리오 x 10회)

▶ Testing Scenario: library / focus
  [1/10] Ludovico Einaudi - Nuvole Bianche (focus_classical)
  [2/10] Ludovico Einaudi - Nuvole Bianche (focus_classical)
  [3/10] Ludovico Einaudi - Nuvole Bianche (focus_classical)
  [4/10] Ludovico Einaudi - Nuvole Bianche (focus_classical)
  [5/10] Ludovico Einaudi - Nuvole Bianche (focus_classical)
  [6/10] Ludovico Einaudi - Nuvole Bianche (focus_classical)
  [7/10] Ludovico Einaudi - Nuvole Bianche (focus_classical)
  [8/10] Ludovico Einaudi - Nuvole Bianche (focus_classical)
  [9/10] Ludovico Einaudi - Nuvole Bianche (focus_classical)
  [10/10] Ludovico Einaudi - Nuvole Bianche (focus_classical)
   Tag Consistency: 100.0% (Dominant: focus_classical)
   Vibe Score: 5/5 (The playlist consists entirely of the same song, 'Nuvole Bianche' by Ludovico Einaudi, which ensures complete consistency in mood and style. This song is tagged as 'focus_classical,' fitting perfectly with the context of a library/f

GPT가 제한된 어휘에 맞춰서 primary_tag를 출력함

> 어휘제한이나 GPT자유설계나 큰 차이 없어보나 어휘제한 안 한게 더 높긴함(일관성만 나머지 지표는 어휘제한한게 더 높음)
> 
> 추후 대시보드와 통계분석을 위해 어휘제한으로 결정